<a href="https://colab.research.google.com/github/Siddhi124github/data-science-prac-1-and-2/blob/main/Practical_5_ANOVA_(Analysis_of_Variance).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SHETH L.U.J & SIR M.V COLLEGE

T096 SIDDHI NAIK

ANOVA (Analysis of Variance)
 Perform one-way ANOVA to compare means across multiple groups.
 Conduct post-hoc tests to identify significant differences between group means.

F-TEST – Comparing Variances of Two Groups
 Problem

Two machines produce metal rods. You measure lengths (in cm):

Machine A: [10.1, 9.9, 10.2, 10.3, 10.0]

Machine B: [10.5, 10.7, 10.6, 10.4, 10.8]

Test if the variances of the two machines are equal at 5% significance.

🔹 Hypotheses

H₀: Variance of A = Variance of B

H₁: Variance of A ≠ Variance of B

In [1]:
import numpy as np
from scipy.stats import f

# Data: lengths from two machines
A = np.array([10.1, 9.9, 10.2, 10.3, 10.0])
B = np.array([10.5, 10.7, 10.6, 10.4, 10.8])

print("Machine A data:", A)
print("Machine B data:", B)

# Sample variances
varA = np.var(A, ddof=1)
varB = np.var(B, ddof=1)

print("\nVariance of A:", varA)
print("Variance of B:", varB)

# Put larger variance on top for F-test
if varA > varB:
    F_stat = varA / varB
    df1 = len(A) - 1
    df2 = len(B) - 1
    group_top = "A"
else:
    F_stat = varB / varA
    df1 = len(B) - 1
    df2 = len(A) - 1
    group_top = "B"

print("\nF statistic (larger variance on top, from group {}):".format(group_top), F_stat)
print("df1 (numerator):", df1)
print("df2 (denominator):", df2)

alpha = 0.05

# Critical F (upper tail)
critical_F = f.ppf(1 - alpha/2, df1, df2)  # two-tailed → alpha/2 in each tail
print("Critical F value (two-tailed, 5%):", critical_F)

# Decision
if F_stat > critical_F:
    print("\nReject H0 → Variances are NOT equal.")
else:
    print("\nAccept H0 → Variances are equal.")


Machine A data: [10.1  9.9 10.2 10.3 10. ]
Machine B data: [10.5 10.7 10.6 10.4 10.8]

Variance of A: 0.024999999999999998
Variance of B: 0.025

F statistic (larger variance on top, from group B): 1.0000000000000002
df1 (numerator): 4
df2 (denominator): 4
Critical F value (two-tailed, 5%): 9.60452988472286

Accept H0 → Variances are equal.


Two-way ANOVA is used when:

You have one numeric variable (e.g., score, height, marks)

You have two categorical factors (e.g., Method, Gender)

You want to see:

Does Factor 1 affect the mean?

Does Factor 2 affect the mean?

Do they interact? (Combined effect)

A teacher wants to study how Teaching Method and Gender affect student scores.

There are two teaching methods:

Method A

Method B

And two genders:

Male

Female
The test scores are:

Method	Gender	Score
A	Male	80
A	Male	78
A	Female	85
A	Female	88
B	Male	90
B	Male	92
B	Female	95
B	Female	93

We want to test, at 5% significance:

Does Teaching Method affect score?

Does Gender affect score?

Is there an interaction between Method and Gender?

Hypotheses (clear and separate)
For Method:

H0 (Method): Mean scores for Method A and Method B are the same.

H1 (Method): Mean scores for Method A and Method B are different.

For Gender:

H0 (Gender): Mean scores for Male and Female are the same.

H1 (Gender): Mean scores for Male and Female are different.

For Interaction (Method × Gender):

H0 (Interaction): There is no interaction between Method and Gender.

H1 (Interaction): There is interaction (effect of Method depends on Gender).

In [3]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Create dataset for two-way ANOVA
data = {
    'Score':  [80, 78, 85, 88, 90, 92, 95, 93],
    'Method': ['A','A','A','A','B','B','B','B'],
    'Gender': ['Male','Male','Female','Female','Male','Male','Female','Female']
}

df = pd.DataFrame(data)
print("Data:")
print(df)

print("\nFactors:")
print("Method levels:", df['Method'].unique())
print("Gender levels:", df['Gender'].unique())

print("\nHypotheses:")
print("H0 (Method): Mean scores are same for Method A and Method B.")
print("H0 (Gender): Mean scores are same for Male and Female.")
print("H0 (Interaction): No interaction between Method and Gender.")

# Build two-way ANOVA model
# Score ~ Method + Gender + Method:Gender (main effects + interaction)
model = ols('Score ~ Method + Gender + Method:Gender', data=df).fit()

anova_table = sm.stats.anova_lm(model, typ=2)

print("\n--- TWO-WAY ANOVA TABLE ---")
print(anova_table)

# Extract p-values
p_method = anova_table.loc['Method', 'PR(>F)']
p_gender = anova_table.loc['Gender', 'PR(>F)']
p_interaction = anova_table.loc['Method:Gender', 'PR(>F)']

alpha = 0.05

print("\n--- DECISIONS (alpha = 0.05) ---")

# Method
print("\nMethod:")
print("p-value (Method):", p_method)
if p_method < alpha:
    print("Reject H0 (Method) → Teaching Method affects score.")
else:
    print("Accept H0 (Method) → Teaching Method does NOT affect score.")

# Gender
print("\nGender:")
print("p-value (Gender):", p_gender)
if p_gender < alpha:
    print("Reject H0 (Gender) → Gender affects score.")
else:
    print("Accept H0 (Gender) → Gender does NOT affect score.")

# Interaction
print("\nInteraction (Method:Gender):")
print("p-value (Interaction):", p_interaction)
if p_interaction < alpha:
    print("Reject H0 (Interaction) → There IS interaction between Method and Gender.")
else:
    print("Accept H0 (Interaction) → NO interaction between Method and Gender.")


Data:
   Score Method  Gender
0     80      A    Male
1     78      A    Male
2     85      A  Female
3     88      A  Female
4     90      B    Male
5     92      B    Male
6     95      B  Female
7     93      B  Female

Factors:
Method levels: ['A' 'B']
Gender levels: ['Male' 'Female']

Hypotheses:
H0 (Method): Mean scores are same for Method A and Method B.
H0 (Gender): Mean scores are same for Male and Female.
H0 (Interaction): No interaction between Method and Gender.

--- TWO-WAY ANOVA TABLE ---
                sum_sq   df          F    PR(>F)
Method         190.125  1.0  72.428571  0.001046
Gender          55.125  1.0  21.000000  0.010164
Method:Gender   10.125  1.0   3.857143  0.121004
Residual        10.500  4.0        NaN       NaN

--- DECISIONS (alpha = 0.05) ---

Method:
p-value (Method): 0.0010456299864771137
Reject H0 (Method) → Teaching Method affects score.

Gender:
p-value (Gender): 0.010163649889538709
Reject H0 (Gender) → Gender affects score.

Interaction (Method:

One-Way ANOVA?

Use one-way ANOVA when:

You have one numeric variable (e.g., marks, score, height)

You have one factor with 3 or more groups

You want to check:

“Are all group means same, or is at least one different?”

2️ Problem Statement (proper)

A teacher uses three different teaching methods and records student scores as follows:

Group A (Method A):
78, 82, 75, 80

Group B (Method B):
85, 88, 84, 90

Group C (Method C):
70, 72, 68, 74

At 5% significance, test whether all three teaching methods have the same mean score.

If not, find which groups are different using a post-hoc test.

3️ Hypotheses

H₀ (Null):
μA = μB = μC
(All three group means are equal.)

H₁ (Alternate):
At least one group mean is different.

This is a one-way ANOVA with 3 groups.

In [4]:
import numpy as np
from scipy.stats import f_oneway
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Marks/scores for three teaching methods
A = np.array([78, 82, 75, 80])   # Method A
B = np.array([85, 88, 84, 90])   # Method B
C = np.array([70, 72, 68, 74])   # Method C

print("Group A (Method A):", A)
print("Group B (Method B):", B)
print("Group C (Method C):", C)

# Means of each group
meanA = np.mean(A)
meanB = np.mean(B)
meanC = np.mean(C)

print("\nMean of Group A:", meanA)
print("Mean of Group B:", meanB)
print("Mean of Group C:", meanC)

print("\nHypotheses:")
print("H0: All group means are equal (μA = μB = μC).")
print("H1: At least one group mean is different.")

# One-way ANOVA
F_stat, p_value = f_oneway(A, B, C)

print("\n--- ONE-WAY ANOVA RESULT ---")
print("F statistic:", F_stat)
print("p-value:", p_value)

alpha = 0.05

if p_value < alpha:
    print("Decision: Reject H0 → At least one group mean is different.")
else:
    print("Decision: Accept H0 → All group means are the same.")

# Post-hoc test (Tukey HSD) only if ANOVA is significant
if p_value < alpha:
    print("\n--- Tukey HSD Post-hoc Test ---")
    # Combine all scores into one array
    scores = np.concatenate([A, B, C])
    # Create labels for each group
    labels = (['A'] * len(A)) + (['B'] * len(B)) + (['C'] * len(C))

    tukey = pairwise_tukeyhsd(endog=scores, groups=labels, alpha=alpha)
    print(tukey)
else:
    print("\nPost-hoc test not needed because H0 was accepted (means are same).")


Group A (Method A): [78 82 75 80]
Group B (Method B): [85 88 84 90]
Group C (Method C): [70 72 68 74]

Mean of Group A: 78.75
Mean of Group B: 86.75
Mean of Group C: 71.0

Hypotheses:
H0: All group means are equal (μA = μB = μC).
H1: At least one group mean is different.

--- ONE-WAY ANOVA RESULT ---
F statistic: 32.12589928057548
p-value: 7.987465152909252e-05
Decision: Reject H0 → At least one group mean is different.

--- Tukey HSD Post-hoc Test ---
 Multiple Comparison of Means - Tukey HSD, FWER=0.05  
group1 group2 meandiff p-adj   lower    upper   reject
------------------------------------------------------
     A      B      8.0 0.0071   2.5138  13.4862   True
     A      C    -7.75 0.0085 -13.2362  -2.2638   True
     B      C   -15.75 0.0001 -21.2362 -10.2638   True
------------------------------------------------------


How to Read the Result

From ANOVA:

If p-value < 0.05 → Reject H₀ → all means are not same

If p-value ≥ 0.05 → Accept H₀ → all means are same

From Tukey:

It will show pairs:

A vs B

A vs C

B vs C

For each pair it tells if difference is significant.